In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Mehdi\\Desktop\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Mehdi\\Desktop\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
           
        )

        return model_evaluation_config


In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch
import pandas as pd
from tqdm import tqdm

d:\anaconda\envs\textS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-10-16 17:31:55,963 : INFO: config: PyTorch version 2.4.1+cu118 available.]


In [9]:
import evaluate
import torch
from tqdm import tqdm
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk

class ModelEvaluation:
    def __init__(self, config):
        self.config = config
    
    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        """Split the dataset into smaller batches for processing."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]
    
    def calculate_metric_on_test_ds(self, dataset, model, tokenizer, 
                                    batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                                    column_text="article", column_summary="highlights"):
        
        # Initialize lists to store predictions and references
        predictions = []
        references = []
        
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024, truncation=True, 
                               padding="max_length", return_tensors="pt").to(device)
            
            summaries = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"], 
                length_penalty=0.8, num_beams=8, max_length=128
            )
            
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                                  clean_up_tokenization_spaces=True) 
                                 for s in summaries]
            
            predictions.extend(decoded_summaries)
            references.extend(target_batch)
        
        # Calculate ROUGE scores using the `evaluate` library
        rouge = evaluate.load('rouge')
        score = rouge.compute(predictions=predictions, references=references,
                              rouge_types=["rouge1", "rouge2", "rougeL", "rougeLsum"], use_aggregator=True)
        
        return score

    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        
        # Load dataset
        dataset = load_from_disk(self.config.data_path)
        
        # Compute ROUGE scores on a subset
        score = self.calculate_metric_on_test_ds(
            dataset['test'][0:10], model, tokenizer, batch_size=2, 
            column_text='dialogue', column_summary='summary'
        )

        # Create a DataFrame and save the results
        rouge_dict = {key: [score[key]] for key in score.keys()}
        df = pd.DataFrame(rouge_dict)
        df.to_csv(self.config.metric_file_name, index=False)


In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2024-10-16 17:31:56,751 : INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-16 17:31:56,753 : INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-16 17:31:56,754 : INFO: common: Directory: artifacts already exists]
[2024-10-16 17:31:56,755 : INFO: common: Directory: artifacts/model_evaluation already exists]


100%|██████████| 5/5 [00:22<00:00,  4.40s/it]


[2024-10-16 17:32:26,449 : INFO: rouge_scorer: Using default tokenizer.]
